<a href="https://colab.research.google.com/github/gkdivya/NLP_Notebooks/blob/main/Session7/Sentiment_Analysis_using_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [95]:
import os
import sys

import pandas


def get_phrase_sentiments(base_directory):
    def group_labels(label):
        if label in ["very negative", "negative"]:
            return "negative"
        elif label in ["positive", "very positive"]:
            return "positive"
        else:
            return "neutral"

    dictionary = pandas.read_csv(os.path.join(base_directory, "dictionary.txt"), sep="|")
    dictionary.columns = ["phrase", "id"]
    dictionary = dictionary.set_index("id")

    sentiment_labels = pandas.read_csv(os.path.join(base_directory, "sentiment_labels.txt"), sep="|")
    sentiment_labels.columns = ["id", "sentiment"]
    sentiment_labels = sentiment_labels.set_index("id")

    phrase_sentiments = dictionary.join(sentiment_labels)

    phrase_sentiments["fine"] = pandas.cut(phrase_sentiments.sentiment, [0, 0.2, 0.4, 0.6, 0.8, 1.0],
                                           include_lowest=True,
                                           labels=["very negative", "negative", "neutral", "positive", "very positive"])
    phrase_sentiments["coarse"] = phrase_sentiments.fine.apply(group_labels)
    return phrase_sentiments


def get_sentence_partitions(base_directory):
    sentences = pandas.read_csv(os.path.join(base_directory, "datasetSentences.txt"), index_col="sentence_index",
                                sep="\t")
    splits = pandas.read_csv(os.path.join(base_directory, "datasetSplit.txt"), index_col="sentence_index")
    return sentences.join(splits).set_index("sentence")


def partition(base_directory):
    phrase_sentiments = get_phrase_sentiments(base_directory)
    sentence_partitions = get_sentence_partitions(base_directory)
    # noinspection PyUnresolvedReferences
    data = phrase_sentiments.join(sentence_partitions, on="phrase")
    data["splitset_label"] = data["splitset_label"].fillna(1).astype(int)
    data["phrase"] = data["phrase"].str.replace(r"\s('s|'d|'re|'ll|'m|'ve|n't)\b", lambda m: m.group(1))
    #return data.groupby("splitset_label")
    return data


In [96]:

base_directory = "https://raw.githubusercontent.com/gkdivya/NLP_Notebooks/main/7.SentimentAnalysisUsingLSTM/stanfordSentimentTreebank/stanfordSentimentTreebank/"
df = partition(base_directory)
df.count()

phrase            239245
sentiment         239245
fine              239245
coarse            239245
splitset_label    239245
dtype: int64

In [97]:
df.loc[6]

phrase            ' ( the cockettes ) provides a window into a s...
sentiment                                                   0.41667
fine                                                        neutral
coarse                                                      neutral
splitset_label                                                    1
Name: 6, dtype: object

In [98]:
df.shape

(239245, 5)

In [99]:
#df = df.drop(columns=['sentiment', 'fine', 'coarse'])

In [100]:
df.head(5)

,phrase,sentiment,fine,coarse,splitset_label
id,,,,,
22935,! ',0.52778,neutral,neutral,1
18235,! '',0.50000,neutral,neutral,1
179257,! Alas,0.44444,neutral,neutral,1
22936,! Brilliant,0.86111,very positive,positive,1
40532,! Brilliant !,0.93056,very positive,positive,1


In [101]:
df.fine.value_counts()

neutral          119451
positive          50152
negative          43032
very positive     15256
very negative     11354
Name: fine, dtype: int64

In [102]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [107]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

stopwords = set(stopwords.words('english') + ['AT_USER','URL'])
    
def processTweet(tweet):
    # tweet is the text we will pass for preprocessing
    # convert passed tweet to lower case 
    tweet = str(tweet).lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    tweet = re.sub(r'#([\s]+)', r'\1', tweet) # remove the # space
    #tweet = re.sub('#^[0-9]*$', r'\1', tweet) # remove # and numbers

    # use work_tokenize imported above to tokenize the tweet
    tweet =  word_tokenize(tweet)
    return [word for word in tweet if word not in stopwords or word not in list(punctuation)]

In [108]:
processed=[]

for phrase in df['phrase']:
    # process all tweets using processTweet function above - store in variable 'cleaned' 
    cleaned=processTweet(phrase)
    processed.append(' '.join(cleaned))

In [111]:
df = df[df['phrase'].map(len) > 4]  


In [112]:
df.head(20)

,phrase,sentiment,fine,coarse,splitset_label
id,,,,,
179257,! Alas,0.44444,neutral,neutral,1
22936,! Brilliant,0.86111,very positive,positive,1
40532,! Brilliant !,0.93056,very positive,positive,1
22937,! Brilliant ! ',1.00000,very positive,positive,1
60624,! C'mon,0.47222,neutral,neutral,1
13402,! Gollum's ` performance ' is incredible,0.76389,positive,positive,1
179258,"! Oh , look at that clever angle ! Wow , a jum...",0.27778,negative,negative,1
140882,! Romething,0.50000,neutral,neutral,1
179259,! Run,0.43056,neutral,neutral,1


In [113]:
df = df[~df.phrase.str.contains("#")]

In [114]:
df.count()

phrase            236135
sentiment         236135
fine              236135
coarse            236135
splitset_label    236135
dtype: int64

In [115]:
df.fine.value_counts()

neutral          116788
positive          49944
negative          42825
very positive     15243
very negative     11335
Name: fine, dtype: int64

In [116]:
df.coarse.value_counts()

neutral     116788
positive     65187
negative     54160
Name: coarse, dtype: int64

In [122]:
df.sample(50)

,phrase,sentiment,fine,coarse,splitset_label
id,,,,,
176682,too random and inconclusive,0.138890,very negative,negative,1
117173,"come ,",0.500000,neutral,neutral,1
31206,especially Lee Ross's turn,0.500000,neutral,neutral,1
129840,over-familiarity,0.347220,negative,negative,1
114970,as its young-guns cast,0.500000,neutral,neutral,1
190408,a cruel deception carried out by men of margin...,0.138890,very negative,negative,1
32248,handily,0.611110,positive,positive,1
58029,the awkwardness,0.458330,neutral,neutral,1
159119,filmmaking scene,0.500000,neutral,neutral,1


In [123]:
def fine_to_label(fine):
  if fine == 'very negative':
    return 0
  elif fine == 'negative':
    return 1
  elif fine == 'neutral':
    return 2
  elif fine == 'positive':
    return 3
  elif fine == 'very positive':
    return 4

df['label'] = df.apply(lambda row: fine_to_label(row['fine']), axis=1)

In [125]:
df.head(5)

,phrase,sentiment,fine,coarse,splitset_label,label
id,,,,,,
179257,! Alas,0.44444,neutral,neutral,1,2
22936,! Brilliant,0.86111,very positive,positive,1,4
40532,! Brilliant !,0.93056,very positive,positive,1,4
22937,! Brilliant ! ',1.00000,very positive,positive,1,4
60624,! C'mon,0.47222,neutral,neutral,1,2


## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [247]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [248]:
Phrase = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [249]:
fields = [('phrase', Phrase),('label',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [250]:
df.shape[0]

236135

In [254]:
#df = df.drop(columns=['sentiment', 'fine', 'coarse', 'splitset_label'])
df.reset_index(drop=True, inplace=True)

In [255]:
df.head(2)

,phrase,label
0,! Alas,2
1,! Brilliant,4


In [256]:
example = [data.Example.fromlist([df.phrase[i],df.label[i]], fields) for i in range(df.shape[0])] 

In [257]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [258]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [284]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [ ]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [ ]:
#Convert text to pd dataframe
def train_todf(train):
  Df = pd.DataFrame(columns=['phrase','label'])
  for t in train:
    text  = " ".join(vars(t)['phrase'])
    label   = vars(t)['label']
    print(text)
    print(random_insertion)
    #Df = Df.append({'phrase':text, 'label':label}, ignore_index = True)
  #return Df

In [293]:
text  = " ".join(vars(valid.examples[0])['phrase'])
#label   = vars(train.example[0][0])['label']
print(text)
print(random_insertion(text,3))

is that rare animal known as ' a perfect family film , ' because it 's about family .


NameError: ignored

In [259]:
(len(train), len(valid))

(200715, 35420)

An example from the dataset:

In [260]:
print(train[0])

In [261]:
vars(train.examples[0])

{'label': 3,
 'phrase': [',',
  'the',
  'film',
  'is',
  'a',
  'hilarious',
  'adventure',
  'and',
  'I',
  'shamelessly',
  'enjoyed',
  'it',
  '.']}

In [262]:
vars(valid.examples[0])

{'label': 4,
 'phrase': ['is',
  'that',
  'rare',
  'animal',
  'known',
  'as',
  "'",
  'a',
  'perfect',
  'family',
  'film',
  ',',
  "'",
  'because',
  'it',
  "'s",
  'about',
  'family',
  '.']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [263]:
Phrase.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [264]:
print('Size of input vocab : ', len(Phrase.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Phrase.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  20816
Size of label vocab :  5
Top 10 words appreared repeatedly : [('the', 64950), (',', 60043), ('a', 46547), ('of', 44363), ('and', 44236), ('.', 32370), ('to', 31621), ('-', 30906), ("'s", 23965), ('is', 19412)]
Labels :  defaultdict(<function _default_unk_index at 0x7fe815720488>, {2: 0, 3: 1, 1: 2, 4: 3, 0: 4})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [265]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [266]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.phrase),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [267]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Phrase.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [268]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [276]:
# Define hyperparameters
size_of_vocab = len(Phrase.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [277]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(20816, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 6,486,905 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [278]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [279]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        print(batch)
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        phrase, phrase_lengths = batch.phrase 
        
        # convert to 1D tensor
        predictions = model(phrase, phrase_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [280]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            phrase, phrase_lengths = batch.phrase
            
            # convert to 1d tensor
            predictions = model(phrase, phrase_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [281]:
print(train_iterator)

**Let's Train and Evaluate**

In [282]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

Streaming output truncated to the last 5000 lines.

[torchtext.data.batch.Batch of size 32]
	[.phrase]:('[torch.LongTensor of size 32x5]', '[torch.LongTensor of size 32]')
	[.label]:[torch.LongTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.phrase]:('[torch.LongTensor of size 32x2]', '[torch.LongTensor of size 32]')
	[.label]:[torch.LongTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.phrase]:('[torch.LongTensor of size 32x2]', '[torch.LongTensor of size 32]')
	[.label]:[torch.LongTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.phrase]:('[torch.LongTensor of size 32x4]', '[torch.LongTensor of size 32]')
	[.label]:[torch.LongTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.phrase]:('[torch.LongTensor of size 32x8]', '[torch.LongTensor of size 32]')
	[.label]:[torch.LongTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.phrase]:('[torch.LongTensor of size 32x5]', '[torch.LongTensor of size 32]')
	[.label]:[torch.LongTe

KeyboardInterrupt: ignored

## Model Testing

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_phrase(phrase):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the phrase 
    tokenized = [tok.text for tok in nlp.tokenizer(phrase)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [283]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library google_trans_new for this purpose. 

In [ ]:
#!pip install google_trans_new

In [ ]:
import random
import google_trans_new
from google_trans_new import google_translator

translator = google_translator()
sentence = ['The dog slept on the rug']

available_langs = list(google_trans_new.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {google_trans_new.LANGUAGES[trans_lang]}")
print(translator)
translations = translator.translate(sentence, lang_tgt=trans_lang) 
print(translations)
#t_text = [t.text for t in translations]
#print(t_text)

translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
#en_text = [t.text for t in translations_en_random]
print(translations_en_random)